In [1]:
%load_ext autoreload
%autoreload

import os, sys

modules = os.path.abspath(os.path.join("..", "src"))
sys.path.append(modules)

import polars as pl
from transformers import AutoModel, AutoTokenizer
import nlp_utils, keywords_utils, load_utils, run_analysis
from config import *

/home/msalvetti/notebooks_2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-20 10:48:09.816187: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Clean data and merge data

## Firm database
Table with 
|Firm ID|Firm name|url main page|canton|employees|NACE code|cat website|cat patent| n kws web| n kws pat

In [ ]:
data = pl.read_csv("../data/raw/url_list_employees.csv")
nace = pl.read_csv("../data/raw/nace_list.csv").select("BVD", "NACE")
kws_web = pl.read_parquet("../data/processed/kw_web_counts.parquet")
kws_pat = pl.read_parquet("../data/processed/kw_pat_counts.parquet")

In [165]:
kws_pat.group_by("ID").agg(
    pl.col("cat_pat"),
    pl.col("n_kw_pat"),
    pl.col("n_unique_kw_pat"),
).write_parquet("../data/processed/kw_pat_counts.parquet")

In [168]:
(
    data
    .drop("").rename({"BVD": "ID"})
    .join(kws_web, on = "ID")
    .join(kws_pat, on = ["ID"], how = "full")
    .filter(pl.col("ID").is_not_null())
    .drop("ID_right")
).write_parquet("../data/processed/panel_all_data.parquet")

In [6]:
data = pl.read_parquet("../data/processed/panel_all_data.parquet")

In [ ]:
patents = pl.read_parquet("../data/cleaned/patents.parquet")
# patent_company_lookup = load_utils.load_multiple_files(pattern = PATENTS_DIR + "Appl*.csv", filetype="csv")[:, 1:3].rename({"Applicant(s) BvD ID Number(s)": "ID"})
# patent_company_lookup = patent_company_lookup.with_columns(pl.col("ID").str.split("; ")).explode("ID")

# patent_results = patents.join(patent_company_lookup, on = "Publication number", how = "inner")
# patent_results
patents

Publication number,Title,Abstract,ID
str,str,str,str
"""CN111201132A""","""METHOD FOR MANUFACTURING PLANA…","""本发明涉及一种用于制造面状复合构件的方法，所述面状复合构件具…","""CHCHE102660797"""
"""US2014089501A1""","""METHOD FOR LOCATING A FIELD DE…","""<abstract lang=""en"" load-sourc…","""CHCHE105329554"""
"""CH715544B1""","""DEVICE AND METHOD FOR CLEANING…","""Die vorliegende Erfindung bezi…","""CHCHE191210690"""
"""AU2012314518A1""","""PYRAZOL-4-YL-HETEROCYCLYL-CARB…","""<abstract lang=""en"" load-sourc…","""CHCHE105815381"""
"""DE2403265A1""","""CONSTANT REFERENCE VOLTAGE GEN…","""<abstract lang=""en"" load-sourc…","""CHCHE105416242"""
…,…,…,…
"""WO2017116478A1""","""METHOD AND APPARATUS FOR WHEEL…","""An apparatus and method of ass…","""CHCHE102674351"""
"""MY184353A""","""FLAVOR CAPSULES FOR SMOKING AR…","""<abstract lang=""en"" load-sourc…","""CHCHE105950151"""
"""CA2833935C""","""NOVEL COMPOUNDS AS MODULATORS …","""The present invention provides…","""CHCHE114577507"""


In [4]:
patents_id = patents.group_by("ID").agg(pl.col("Publication number"))

In [9]:
data.join(patents_id, on = "ID")#.write_parquet("../data/data_tobias.parquet")

ID,Name,url,Canton,n_employees,cat_web,n_kw_web,n_unique_kw_web,cat_pat,n_kw_pat,n_unique_kw_pat,Publication number
str,str,str,str,f64,list[str],list[u32],list[u32],list[str],list[u32],list[u32],list[str]
"""CHCHE234402550""","""3A COMPOSITES MOBILITY AG""","""3acompositesmobility.com""","""St. Gallen""",15.0,[null],[null],[null],"[""Advanced composite materials"", null]","[2, null]","[1, null]","[""CN117881587A"", ""WO2023031133A1"", … ""CA3227466A1""]"
"""CHCHE105792739""","""GENOSSENSCHAFT MIGROS AARE""","""aare.migros.ch/de.html""","""Bern""",8511.0,[null],[null],[null],[null],[null],[null],"[""EP4137281A1""]"
"""CHCHE474053474""","""ACROTEC R&D SA""","""acrotec.ch/en""","""Neuchatel""",15.0,[null],[null],[null],"[""Nanoscale materials and manufacturing"", null, … ""Additive manufacturing (including 3D printing)""]","[4, null, … 28]","[1, null, … 4]","[""CH720150B1"", ""WO2022223479A1"", … ""CH718551A2""]"
"""CHCHE106132936""","""ROBT. SCHWARZENBACH & CO AG""","""alexisschwarzenbach.ch""","""Zurich""",15.0,[null],[null],[null],"[null, ""Critical minerals extraction and processing""]","[null, 2]","[null, 1]","[""GB105237A"", ""GB1109686A""]"
"""CHCHE302525568""","""AL-S PHARMA AG""","""alpenpharma.ch""","""Zurich""",15.0,"[""Vaccines and medical countermeasures"", ""AI algorithms and hardware accelerators"", … ""Biological manufacturing""]","[21, 2, … 1]","[3, 1, … 1]",[null],[null],[null],"[""AU2021237769A1"", ""WO2024042250A1"", … ""AU2023329200A1""]"
…,…,…,…,…,…,…,…,…,…,…,…
"""CHCHE102041909""","""ZWICKY & CO AG""","""www.zwicky-areal.ch""","""Zurich""",35.0,[null],[null],[null],"[""nuclear waste management and recycling"", ""Additive manufacturing (including 3D printing)"", null]","[4, 5, null]","[2, 2, null]","[""FR2494311A1"", ""EP0052268A1""]"
"""CHCHE101539242""","""ZWILAG ZWISCHENLAGER WUERENLIN…","""www.zwilag.ch""","""Aargau""",95.0,"[""nuclear energy"", ""nuclear waste management and recycling"", ""Nuclear medicine and radiation therapy""]","[141, 65, 113]","[9, 5, 4]",[null],[null],[null],"[""CY1106495T1"", ""TR200702326T4"", … ""EP1533567A1""]"
"""CHCHE109026265""","""ZEHNDER & SOMMER AG""","""zehndersommer.com""","""Bern""",35.0,"[""Drones swarming and collaborative robots"", ""Advanced magnets and superconductors"", … ""Advanced robotics""]","[138, 1, … 28]","[1, 1, … 4]",[null],[null],[null],"[""US12145818B2"", ""US2024068493A1"", … ""CN117627995A""]"
